# *spaCy* : une bibliothèque pour le traitement du langage naturel

Considérée comme une alternative moderne à la bibliothèque NTLK (*Natural Language ToolKit*), *spaCy* intègre de très nombreux outils état de l’art pour l’analyse de données textuelles, mobilisables simplement, au prix d’une perte des possibilités de personnalisation.

Une fois installée, la bibliothèque se charge avec l’espace de noms `spacy` :

In [ ]:
import spacy

## Comparaison pour la segmentation d’un texte français en mots

Soit la phrase suivante :

In [ ]:
sent = "Elvire, m’as-tu fait un rapport bien sincère ?"

En permettant de paramétrer un outil pour la segmentation en mots basée sur les expressions rationnelles, NLTK offre beaucoup de souplesse et n’effectue aucune hypothèse quant à la langue du texte :

In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer

# a custom tokenizer
tokenizer = RegexpTokenizer(r"\w+")

# tokenization
tokens = tokenizer.tokenize(sent)

# iterate over tokens
for token in tokens:
    print(token, end=" ")

Avec *spaCy*, il est nécessaire de charger un outil polyvalent, entraîné pour le français :

In [ ]:
import spacy

# create a blank object customized for French
nlp = spacy.blank("fr")

# process the document
doc = nlp(sent)

# iterate over tokens
for token in doc:
    if not token.is_punct:
        print(token.text, end=" ")

## Charger un pipeline pré-entraîné

L’un des avantages indéniables de *spaCy* réside dans la possibilité de charger des pipelines pré-entraînés sur de larges corpus étiquetés pour effectuer des prédictions en contexte. Un pipeline peut se concevoir comme un modèle de langue configuré à l’aide de plusieurs transformateurs (tokenisation, lemmatisation, vectorisation…). Pour le français, il en existe quatre :

- *fr_core_news_sm* : données textuelles issues de la presse avec un vocabulaire, une analyse syntaxique et la reconnaissance des entités nommées (15 MB) ;
- *fr_core_news_md* : + vectorisation (20 000 vecteurs uniques de 300 dimensions) (43 MB) ;
- *fr_core_news_lg* : vectorisation plus complète (500 000 vecteurs uniques) (545 MB) ;
- *fr_dep_news_trf* : transformateur basé sur camemBERT.

La méthode `.load()` charge le pipeline demandé :

In [ ]:
nlp = spacy.load("fr_core_news_sm")

![Pipeline](./images/pipeline.svg)

Pour appliquer l’ensemble des transformateurs à un texte, il suffit d’utiliser la fonction `nlp()` :

In [ ]:
text = """C'était à Mégara, faubourg de Carthage, dans les jardins d'Hamilcar.
Les soldats qu'il avait commandés en Sicile se donnaient un grand festin pour célébrer le jour anniversaire de la bataille d'Eryx, et comme le maître était absent et qu'ils se trouvaient nombreux, ils mangeaient et ils buvaient en pleine liberté.
Les capitaines, portant des cothurnes de bronze, s'étaient placés dans le chemin du milieu, sous un voile de pourpre à franges d'or, qui s'étendait depuis le mur des écuries jusqu'à la première terrasse du palais ; le commun des soldats était répandu sous les arbres, où l'on distinguait quantité de bâtiments à toit plat, pressoirs, celliers, magasins, boulangeries et arsenaux, avec une cour pour les éléphants, des fosses pour les bêtes féroces, une prison pour les esclaves.
"""
doc = nlp(text)

L’objet `doc` embarque désormais des structures, accessibles depuis des attributs (p. ex. `.sents` pour les phrases), en fonction des transformateurs utilisés.

## Les propriétés de l’objet `doc`

En règle générale, les propriétés suivies d’un tiret bas (*_*) renvoient une chaîne de caractères ; pour les autres, un entier qui vaut comme identifiant.

Comme les données textuelles sont analysées par un pipeline pré-entraîné, les propriétés affectées aux *tokens* sont issues de prédictions qui peuvent par définition se révéler inexactes. Le résultat de l’évaluation des modèles pour chaque tâche est consultable sur [le site officiel](https://spacy.io/models/). Pour un projet spécifique, gardons en tête qu’il est toujours possible d’ajouter des données annotées au pipeline afin d’améliorer son efficacité.

### Segmentation en tokens

Par défaut, l’objet obtenu est un assemblage de *tokens* avec différents attributs dont `.text` pour leur représentation en chaîne de caractères :

In [ ]:
for token in doc:
    print(token.text, end=" ")

### Segmentation en phrases

Les phrases sont accessibles via l’attribut `.sents` qui héberge un générateur :

In [ ]:
for sent in doc.sents:
    print(sent)

first_sent = list(doc.sents)[0]

### Lemmatisation

Pour accéder aux lemmes, utiliser la propriété `.lemma_` :

In [ ]:
for token in first_sent:
    print(token.text, token.lemma_)

### Analyse morphologique

L’attribut `.pos_` permet d’accéder aux étiquettes en parties du discours du jeu *universal part-of-speech tagset*. Pour une analyse morphologique complète (genre, nombre, mode…), l’attribut `.morph` stocke une chaîne de caractètres formatée :

In [ ]:
for token in first_sent:
    print(token.text, token.pos_, token.morph)

### Analyse en dépendances

Les modèles peuvent également prédire les relations entre les *tokens* d’une phrase. L’attribut `.dep_` donne accès à sa fonction grammaticale quand l’attribut `.head` révèle le sujet auquel il est rattaché :

In [ ]:
for token in first_sent:
    print(token.text, token.dep_, token.head.text)

Un module *displacy* permet en plus d’afficher l’arbre des dépendances :

In [ ]:
from spacy import displacy

displacy.render(first_sent, style="dep", options={ "distance": 100 })

### Reconnaître les entités nommées

La capacité de *spaCy* à repérer des entités nommées (personnes, organisations, événements…) est l’une de ses très grandes forces. Elles sont enregistrées dans une propriété `.ents` qui renvoie un tuple de toutes les entités prédites. Pour chacune d’elles, il est possible d’afficher son contenu et son étiquette :

In [ ]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Le module *displacy* se révèle ici également très efficace pour surligner les entités nommées dans un texte :

In [ ]:
displacy.render(first_sent, style="ent")

### Autres attributs lexicaux

Aux côtés de l’attribut lexical `.text`, on trouve d’autres propriétés utiles :

- `.i` : renvoie l’indice du *token*.
- `.is_alpha` : le *token* est-il constitué de caractères alphabétiques ?
- `.is_punct` : le *token* est-il un signe de ponctuation ?
- `.like_num` : le *token* ressemble-t-il à un nombre ?

In [ ]:
for token in first_sent:
    if not token.like_num and not token.is_punct:
        print(token.text)

## Expliquer les abréviations

Les résultats fournis par les pipelines de *spaCy* le sont sous forme d’abréviations. Pour connaître leur forme étendue, une méthode `.explain()` est attachée directement à l’espace de noms :

In [ ]:
spacy.explain("MISC")